In [52]:
#下面这行代码，是为了把自己编写的代码文件当作一共模块导入，这里是把Utilities文件夹中的plotting.py文件当作python的模块导入，对应的是下面的from plotting import newfig, savefig。路径要随着不同设备的系统做相应的修改
import sys #导入sys模块。sys模块提供了一些变量和函数，用于与 Python解释器进行交互和访问。例如，sys.path 是一个 Python 在导入模块时会查找的路径列表，sys.argv 是一个包含命令行参数的列表，sys.exit() 函数可以用于退出 Python 程序。导入 sys 模块后，你就可以在你的程序中使用这些变量和函数了。
sys.path.insert(0, '../../Utilities/') #在 Python的sys.path列表中插入一个新的路径。sys.path是一个 Python 在导入模块时会查找的路径列表。新的路径'../../Utilities/'相对于当前脚本的路径。当你尝试导入一个模块时，Python 会在 sys.path 列表中的路径下查找这个模块。通过在列表开始位置插入一个路径，你可以让 Python 优先在这个路径下查找模块。这在你需要导入自定义模块或者不在 Python 标准库中的模块时非常有用。

import torch
#collections是python一个内置模块，提供了一些有用的数据结构
from collections import OrderedDict  #这个类是字典dict的一个子类，用于创建有序的字典。普通字典中元素顺序是无序的，在OrderedDict中元素的顺序是有序的，元素的顺序是按照它们被添加到字典中的顺序决定的。

from pyDOE import lhs #`pyDOE`是一个Python库，用于设计实验。它提供了一些函数来生成各种设计，如因子设计、拉丁超立方设计等。`lhs`是库中的一个函数，全名为"Latin Hypercube Sampling"，拉丁超立方采样。这是一种统计方法，用于生成一个近似均匀分布的多维样本点集。它在参数空间中生成一个非常均匀的样本，这对于高维数值优化问题非常有用，因为它可以更好地覆盖参数空间。
import numpy as np
import matplotlib.pyplot as plt
import scipy.io #导入了scipy库中的io模块。scipy.io模块包含了一些用于文件输入/输出的函数，例如读取和写入.mat文件（MATLAB格式）
from scipy.interpolate import griddata #`scipy.interpolate`是`scipy`库中的一个模块，提供了许多插值工具，用于在给定的离散数据点之间进行插值和拟合。`griddata`是这个模块中的一个函数，用于在无规则的数据点上进行插值。

import random

import skopt #用于优化问题的库，特别是机器学习中的超参数优化
from distutils.version import LooseVersion #distutils是Python的一个标准库，用于构建和安装Python包。LooseVersion是一个类，用于比较版本号


from plotting_torch import newfig, savefig #从自定义的plotting_torch.py文件中导入了newfig和savefig函数。这两个函数用于创建和保存图形。这两个函数的定义在plotting_torch.py文件中

from mpl_toolkits.axes_grid1 import make_axes_locatable #`mpl_toolkits.axes_grid1`是`matplotlib`库的一个模块，提供了一些高级的工具来控制matplotlib图形中的坐标轴和颜色条。`make_axes_locatable`是模块中的一个函数，用于创建一个可分割的坐标轴。可以在这个坐标轴的四个方向（上、下、左、右）添加新的坐标轴或颜色条。
import matplotlib.gridspec as gridspec #是`matplotlib`库的一个模块，用于创建一个网格布局来放置子图。在`matplotlib`中可以创建一个或多个子图（subplot），每个子图都有自己的坐标轴，并可以在其中绘制图形。`gridspec`模块提供了一个灵活的方式来创建和放置子图。
import time #一个内置模块，用于处理时间相关的操作。


from tqdm import tqdm #一个快速，可扩展的python进度条库，可以在python长循环中添加一个进度提示信息，用户只需要封装任意的迭代器tqdm(iterator)。

import os
import pickle

import timeit #用于计时和测量小段代码的执行时间
import seaborn as sns  # 导入seaborn库用于绘制密度图
import pandas as pd #一个用于数据操作和分析的库，提供了数据结构和数据分析工具，特别是用于处理表格数据（类似于Excel中的数据表）

import subprocess
import threading

In [53]:
num_gpus = torch.cuda.device_count()
print('Number of available GPUs: ', num_gpus)

for i in range(num_gpus):
    torch.cuda.set_device(i)
    allocated = torch.cuda.memory_allocated()
    reserved = torch.cuda.memory_reserved()
    print('GPU {}: {}, Allocated: {}, Reserved: {}'.format(i, torch.cuda.get_device_name(i), allocated, reserved))

Number of available GPUs:  2
GPU 0: NVIDIA GeForce RTX 4090, Allocated: 104194048, Reserved: 111149056
GPU 1: NVIDIA GeForce RTX 4090, Allocated: 17039360, Reserved: 31457280


In [54]:
torch.cuda.set_device(0) #设置当前使用的GPU设备。这里设置为1号GPU设备（第二块显卡）。

# CUDA support 

#设置pytorch的设备，代表了在哪里执行张量积算，设备可以是cpu或者cuda（gpu），并将这个做运算的设备对象存储在变量device中，后续张量计算回在这个设备上执行
if torch.cuda.is_available():
    device = torch.device('cuda')
else:
    device = torch.device('cpu')

In [55]:
# the deep neural network
class DNN(torch.nn.Module):
    #第一个方法
    def __init__(self, layers):
        super(DNN, self).__init__() #调用父类的__init__方法进行初始化
        
        # parameters
        self.depth = len(layers) - 1 #定义名为depth的属性，表示神经网络的深度，等于层数-1
        
        # set up layer order dict
        self.activation = torch.nn.Tanh #设置激活函数为tanh
         
        layer_list = list() #定义一个空列表layer_list
        for i in range(self.depth - 1):  #循环depth次
            #将每一层（全连接层）添加到layer_list中
            layer_list.append(
                ('layer_%d' % i, torch.nn.Linear(layers[i], layers[i+1]))
            )
            #将每一层的激活函数添加到layer_list中
            layer_list.append(('activation_%d' % i, self.activation()))

        #循环结束后，将最后一层的线性变换添加到layer_list中（激活函数为softmax）
        layer_list.append(
            ('layer_%d' % (self.depth - 1), torch.nn.Linear(layers[-2], layers[-1]))
        )
        #输出层的激活函数为tanh
        layer_list.append(('activation_%d' % (self.depth - 1), torch.nn.Softmax(dim=1)))  

        #然后使用OrderedDict将layer_list中的元素转换为有序字典
        layerDict = OrderedDict(layer_list)
        
        # deploy layers，将layerDict转换为一个神经网络模型，赋值给self.layers
        self.layers = torch.nn.Sequential(layerDict) #layers，长度为2*depth，包含了所有的线性变换和激活函数，奇数索引为线性变换(权重矩阵)，偶数索引为激活函数
    
    #第二个方法，定义了模型的前向传播过程
    def forward(self, x):  #接收输入x
        out = self.layers(x) #将输入x传入神经网络模型self.layers中，得到输出out
        return out #返回输出out
    
    # # 新增方法，获取最后一个隐藏层的输出
    # def hidden_output(self, x):
    #     # 遍历每一层，直到最后一个隐藏层
    #     for i in range(self.depth - 1):
    #         # 获取当前层的线性变换
    #         x = self.layers[i*2](x)
    #         # 获取当前层的激活函数
    #         x = self.layers[i*2 + 1](x)
    #     # 返回最后一个隐藏层的输出
    #     return x

    # 新增方法，循环传递输入
    def hidden_forward(self, x, num_cycles):
        # 将输入传递到最后一个隐藏层，x为最后一个隐藏层的输出
        for i in range(self.depth - 1):
            x = self.layers[i*2](x)
            x = self.layers[i*2 + 1](x)
        
        # 将最后一个隐藏层的输出作为输入传递到第一个隐藏层中，重复num_cycles次
        for _ in range(num_cycles):
            for i in range(1, self.depth - 1): #最开始的x为最后一个隐藏层的输出，然后从第一个隐藏层开始传递
                x = self.layers[i*2](x)
                x = self.layers[i*2 + 1](x)
        hidden_output = x
        
        return hidden_output

In [56]:
# the physics-guided neural network
class PhysicsInformedNN():
    # Initialize the class
    def __init__(self, X_u, u, X_f, layers, lb, ub): #这个类包含的第一个方法__init__，这是一个特殊的方法，也就是这个类的构造函数，用于初始化新创建的对象，接受了几个参数

        mu_x, sigma_x = X_f.mean(0), X_f.std(0) #计算X_u的均值和标准差
        X_u = (X_u - mu_x) / sigma_x #对X_u进行标准化
        X_f = (X_f - mu_x) / sigma_x #对X_f进行标准化

        self.mu_x = torch.tensor(mu_x).float().to(device) #创建一个pytorch张量（数据来源于mu_x），并将其转换为浮点类型，最后将张量移动到指定的设备上
        self.sigma_x = torch.tensor(sigma_x).float().to(device) #创建一个pytorch张量（数据来源于sigma_x），并将其转换为浮点类型，最后将张量移动到指定的设备上

        # boundary conditions
        #将传入的lb和ub参数的值存储在实例中，以便后续使用。这样可以在类的其他方法中通过self.lb和self.ub来访问这些值。
        self.lb = torch.tensor(lb).float().to(device) #创建一个pytorch张量（数据来源于lb），并将其转换为浮点类型，最后将张量移动到指定的设备上
        self.ub = torch.tensor(ub).float().to(device)
        
        # data
        #初边界点数据
        self.X_u = torch.tensor(X_u, requires_grad=True).float().to(device)
        #初边界点真实数据
        self.u = torch.tensor(u).float().to(device)
        #配位点数据
        self.X_f = torch.tensor(X_f, requires_grad=True).float().to(device)
        # #测试点数据
        # self.X_star = torch.tensor(X_star).float().to(device)
        # #测试点真实数据
        # self.u_star = torch.tensor(u_star).float().to(device)
        
        self.layers = layers
        
        # deep neural networks
        self.dnn = DNN(layers).to(device) #创建一个DNN类的实例，传入layers参数来实现神经网络的初始化，然后将这个实例移动到指定的设备上
        
        # optimizers: using the same settings
        #创建优化器，括号内为要优化的参数，使用Adam优化方法
        self.optimizer_Adam = torch.optim.Adam(self.dnn.parameters(), eps = 1e-8) 


        self.iter = 0 #记录迭代次数 

        self.loss_value = [] #创建一个空列表，用于存储损失值

        self.test_error = [] #创建一个空列表，用于存储测试误差
        self.test_error_mse = [] #创建一个空列表，用于存储测试误差
        self.test_error_mae = [] #创建一个空列表，用于存储测试误差


    #主动学习更新配位点数据    
    def update_data(self, X_u, u, X_f):
        mu_x, sigma_x = X_f.mean(0), X_f.std(0)
        X_u = (X_u - mu_x) / sigma_x
        X_f = (X_f - mu_x) / sigma_x

        self.mu_x = torch.tensor(mu_x).float().to(device)
        self.sigma_x = torch.tensor(sigma_x).float().to(device)
        
        self.X_u = torch.tensor(X_u, requires_grad=True).float().to(device)
        self.u = torch.tensor(u).float().to(device)
        self.X_f = torch.tensor(X_f, requires_grad=True).float().to(device)
        
    #定义了一个名为net_u的函数/方法，用于计算神经网络的输出。这个方法接受两个参数，分别是x和t，其中x是输入数据，t是时间数据。最后返回神经网络的输出。     
    def net_u(self, x):  
        u = self.dnn(x)  #（第一个参数将输入的两个参数x和t在第二个维度（列）上进行拼接，形成一个新的张量）调用DNN，根据两个参数权重和偏置，以及新得到的张量，计算神经网络的输出u
        return u
    
    #定义了一个名为net_f的函数/方法，用于计算论文中的f。这个方法接受两个参数，分别是x和t，其中x是输入数据，t是时间数据。最后返回计算得到的f。
    def net_f(self, x):
        """ The pytorch autograd version of calculating residual """
        u = self.net_u(x) #调用上面的net_u函数，计算神经网络的输出u
        u1 = u[:, 0:1] #取u的第一列，对应状态1
        u2 = u[:, 1:2] #取u的第二列，对应状态2
        u3 = u[:, 2:3] #取u的第三列，对应状态3

        u_x_1 = torch.autograd.grad(
            u1, x, #输入的张量，要计算u1关于x的导数
            grad_outputs=torch.ones_like(u1), #生成一个与u1形状相同，所有元素均为1的张量，这个参数用于指定向量-雅可比积的像两部分
            retain_graph=True, #表示计算完梯度之后保留计算图若需要多次计算梯度，则需要设置改参数为True
            create_graph=True #创建梯度的计算图，使我们能够计算高阶导数
        )[0] #这个函数的返回值是一个元组，其中包含了每个输入张量的梯度。这里只关心第一个输入张量u1的梯度，所以我们使用[0]来获取这个梯度。？？？？又说只有一个梯度
        u_x_2 = torch.autograd.grad(
            u2, x, #输入的张量，要计算u2关于x的导数
            grad_outputs=torch.ones_like(u2), #生成一个与u2形状相同，所有元素均为1的张量，这个参数用于指定向量-雅可比积的像两部分
            retain_graph=True, #表示计算完梯度之后保留计算图若需要多次计算梯度，则需要设置改参数为True
            create_graph=True #创建梯度的计算图，使我们能够计算高阶导数
        )[0] #这个函数的返回值是一个元组，其中包含了每个输入张量的梯度。这里只关心第一个输入张量u2的梯度，所以我们使用[0]来获取这个梯度。？？？？又说只有一个梯度
        u_x_3 = torch.autograd.grad(
            u3, x, #输入的张量，要计算u3关于x的导数
            grad_outputs=torch.ones_like(u3), #生成一个与u3形状相同，所有元素均为1的张量，这个参数用于指定向量-雅可比积的像两部分
            retain_graph=True, #表示计算完梯度之后保留计算图若需要多次计算梯度，则需要设置改参数为True
            create_graph=True #创建梯度的计算图，使我们能够计算高阶导数
        )[0] #这个函数的返回值是一个元组，其中包含了每个输入张量的梯度。这里只关心第一个输入张量u3的梯度，所以我们使用[0]来获取这个梯度。？？？？又说只有一个梯度

        #归一化
        u_x_1 = u_x_1 / self.sigma_x
        u_x_2 = u_x_2 / self.sigma_x
        u_x_3 = u_x_3 / self.sigma_x
        
        #计算残差
        xx = x * self.sigma_x + self.mu_x
        residual_1 = u_x_1-(-2.572e-4*xx*u1)
        residual_2 = u_x_2-(1.12e-4*xx*u1-2.012e-4*xx*u2)
        residual_3 = u_x_3-(1.452e-4*xx*u1+2.012e-4*xx*u2)    

        loss_1 = torch.mean(residual_1 ** 2)
        loss_2 = torch.mean(residual_2 ** 2)
        loss_3 = torch.mean(residual_3 ** 2)

        loss_f = loss_1 + loss_2 + loss_3
        return loss_f, residual_1+residual_2+residual_3
    
      
    
    def train(self, nIter):


        #使用Adam优化器优化nIter次
        for epoch in tqdm(range(nIter), desc='Adam'):
            self.dnn.train() #将神经网络设置为训练模式而不是评估模式
            u_pred = self.net_u(self.X_u) #调用之前定义的函数，传入初值得到神经网络的输出u

            loss_u = torch.mean((self.u - u_pred) ** 2) #计算初值的损失函数
            loss_f, _ = self.net_f(self.X_f) #计算残差的损失函数
            

            loss = loss_u + loss_f
            
            # Backward and optimize
            self.optimizer_Adam.zero_grad() #清除该优化器之前计算的梯度（在PyTorch中，梯度会累积，所以在每次新的优化迭代之前，我们需要清除之前的梯度）
            loss.backward() #被调用以计算损失函数关于神经网络参数的梯度。这个梯度将被用于优化器来更新神经网络参数
            self.optimizer_Adam.step()  #使用之前的优化器self.optimizer_Adam，调用step方法(执行一步优化算法)，传入损失函数self.loss_func，进行优化
            
            #record the loss value
            self.loss_value.append(loss) #将计算得到的loss值添加到self.loss_value列表中

            # # record the test error
            # self.dnn.eval() #将神经网络切换为评估模式
            # with torch.no_grad():
            #     u_real_pred = self.net_u(self.x_star, self.t_star) #调用之前定义的函数，传入参数得到神经网络的输出u


            # error_test = torch.norm(self.u_star-u_real_pred,2)/torch.norm(self.u_star,2)

            # self.test_error.append(error_test)

            # # 计算 MAE和MSE
            # mae = torch.mean(torch.abs(self.u_star - u_real_pred))
            # mse = torch.mean((self.u_star - u_real_pred) ** 2)
            # # 记录 MAE 和 MSE
            # self.test_error_mae.append(mae)
            # self.test_error_mse.append(mse)


  

    def predict(self, X):
        x = torch.tensor(X, requires_grad=True).float().to(device) #从输入中得到x和t（第一列和第二列），是张量，需要计算梯度，转换为浮点数类型，并将张量移动到指定设备上
        mu, sigma = self.mu_x, self.sigma_x
        x = (x - mu) / sigma

        self.dnn.eval() #将神经网络切换为评估模式
        u = self.net_u(x) #调用之前定义的函数得到神经网络的输出u,以及f
        _, f = self.net_f(x)
        u = u.detach().cpu().numpy() #将张量u和f先从计算图中分离出来，然后转换为numpy数组，最后将这个数组移动到cpu上
        f = f.detach().cpu().numpy()
        return u, f
    

    def hidden_predict(self, x, num_cycles):
        x = torch.tensor(x, requires_grad=True).float().to(device) #从输入中得到x和t（第一列和第二列），是张量，需要计算梯度，转换为浮点数类型，并将张量移动到指定设备上
        mu, sigma = self.mu_x, self.sigma_x
        x = (x - mu) / sigma

        self.dnn.eval()
        hidden_output = self.dnn.hidden_forward(x, num_cycles)
        hidden_output = hidden_output.detach().cpu().numpy()
        return hidden_output

In [57]:
#定义设置随机数种子的函数，第一个参数seed表示种子；第二个参数用来设置CUDA的卷积操作是否确定性，默认为False，表示没有确定性
def set_seed(seed):
    # torch.manual_seed(seed) #设置pytorch的CPU随机数生成器的种子
    # torch.cuda.manual_seed_all(seed) #设置putorch的所有GPU随机数生成器的种子
    # np.random.seed(seed) #设置numpy的随机数生成器的种子
    # random.seed(seed) #设置python的内置随机数生成器的种子
    # torch.backends.cudnn.deterministic = deterministic #True会让CUDA的卷积操作变得确定性，即对于相同的输入，每次运行会得到相同的结果，False则相反
    """
    设置PyTorch的随机种子, 用于生成随机数. 通过设置相同的种子, 可以确保每次运行时生成的随机数序列相同
    """
    torch.manual_seed(seed)
 
    """
    设置PyTorch在所有可用的CUDA设备上的随机种子. 如果在使用GPU进行计算, 这个设置可以确保在不同的GPU上生成的随机数序列相同
    """
    torch.cuda.manual_seed_all(seed)
 
    """
    设置PyTorch在当前CUDA设备上的随机种子. 它与上一行代码的作用类似, 但只影响当前设备
    """
    torch.cuda.manual_seed(seed)
 
    """
    设置NumPy的随机种子, 用于生成随机数. 通过设置相同的种子，可以确保在使用NumPy的随机函数时生成的随机数序列相同
    """
    np.random.seed(seed)
    
    """
    设置Python内置的随机函数的种子. Python的random模块提供了许多随机函数, 包括生成随机数、打乱列表等. 通过设置相同的种子, 可以确保使用这些随机函数时生成的随机数序列相同
    """
    random.seed(seed)
    
    """
    设置Python的哈希种子 (哈希函数被广泛用于数据结构 (如字典和集合) 的实现，以及一些内部操作 (如查找和比较)). 通过设置相同的种子, 可以确保在不同的运行中生成的哈希结果相同
    """
    # os.environ["PYTHONHASHSEED"] = str(seed)
    
    """
    该设置确保每次运行代码时, cuDNN的计算结果是确定性的, 即相同的输入会产生相同的输出, 这是通过禁用一些非确定性的算法来实现的, 例如在卷积操作中使用的算法. 这样做可以保证模型的训练和推理在相同的硬件和软件环境下是可复现的, 即每次运行代码时的结果都相同. 但是, 这可能会导致一些性能上的损失, 因为禁用了一些优化的非确定性算法
    """
    torch.backends.cudnn.deterministic = True
    
    """
    该设置禁用了cuDNN的自动优化过程. 当它被设置为False时, PyTorch不会在每次运行时重新寻找最优的算法配置, 而是使用固定的算法配置. 这样做可以确保每次运行代码时的性能是一致的, 但可能会导致一些性能上的损失
    """
    torch.backends.cudnn.benchmark = False



In [58]:
# 定期采样GPU使用情况的函数
def sample_gpu_usage(interval, gpu_usage_list, stop_event, gpu_index): #interval是采样间隔时间
    while not stop_event.is_set():
        gpu_usage = subprocess.check_output(
            ['nvidia-smi', '-i', str(gpu_index), '--query-gpu=utilization.gpu,memory.used', '--format=csv,noheader,nounits']
        ).decode('utf-8').strip().split('\n')[0].split(', ')
        gpu_usage = [int(x) for x in gpu_usage]
        gpu_usage_list.append(gpu_usage)
        time.sleep(interval)

# 模拟训练函数
def train_model(model, Iter, sample_interval=1, gpu_index=0): #这里gpu_index代表GPU的索引，sample_interval代表采样间隔时间几s
    # 用于存储GPU使用情况的列表
    gpu_usage_list = []
    stop_event = threading.Event()

    # 启动一个线程定期采样GPU使用情况
    sampling_thread = threading.Thread(target=sample_gpu_usage, args=(sample_interval, gpu_usage_list, stop_event, gpu_index))
    sampling_thread.start()

    start_time = time.time()  # 获取当前时间
    # 训练模型
    model.train(Iter)
    end_time = time.time() # 获取当前时间
    training_time = end_time - start_time # 计算训练时间

    # 停止采样线程
    stop_event.set()
    sampling_thread.join()

    # 计算平均GPU使用情况
    avg_gpu_usage = np.mean(gpu_usage_list, axis=0)

    return training_time, avg_gpu_usage

In [59]:
#basic PINN
#RAR-G方法，对1000个点，先选择10个点训练500次，然后每500次迭代重采样100个点，选出其中残差最大的10个点添加到训练点中；最后总共有1000个点，共训练10000次
seeds = [0, 1, 12, 33, 123, 321, 1234, 4321, 12345, 54321] #生成10个随机种子
# seeds = [0]
#设置噪声水平为0
noise = 0.0        

#读取真实解(基于MCS方法得到的解)
MCS_solution_df = pd.read_csv('MCS_non_homo_300.csv')
MCS_solution_df.columns = ['time', 'state_0', 'state_1', 'state_2']

MCS_solution_df.drop(labels='time', inplace=True, axis = 1)


N_u = 1
N_f = 300

#定义一个列表layers，其中包含了神经网络的层数和每一层的神经元数量
layers = [1, 50, 50, 3]

lb  = np.array([0.0])              # t的左边界
ub  = np.array([300.0])          # t的右边界

# Define the initial MSS state
X_u = np.array([[0]])
u = np.array([[1,0,0]])          # 初始真实状态
# X_f = np.linspace(lb, ub, N_f)     # 配位点位置，0~60000的5000个点

PINN_training_time = [] #创建一个空列表，用于存储PINN训练时间
GPU_usage = [] #创建一个空列表，用于存储GPU使用率
GPU_memory = [] #创建一个空列表，用于存储GPU显存占用

i = 0 #初始化i为0



for seed in seeds:
    set_seed(seed) #设置随机数种子

    X_f = lb + (ub - lb) * lhs(1, N_f) #生成N_f个随机点
    # X_f = np.linspace(lb, ub, N_f)     # 配位点位置，0~60000的5000个点
    nIter = 150000 #设置迭代次数为10000

    #创建PINN模型并输入各种参数     
    model = PhysicsInformedNN(X_u, u, X_f, layers, lb, ub)
    #开始训练


    training_time, avg_gpu_usage = train_model(model, Iter = nIter, sample_interval=1)
    print(f"训练时间: {training_time:.2f} 秒")
    print(f"平均GPU使用率: {avg_gpu_usage[0]:.2f}%")
    print(f"平均GPU显存使用量: {avg_gpu_usage[1]:.2f}MiB")

    PINN_training_time.append(training_time)
    GPU_usage.append(avg_gpu_usage[0])
    GPU_memory.append(avg_gpu_usage[1])

    i+=1 #i加1
    print(f'当前为第{i}次循环，种子为{seed}')

    
    


Adam: 100%|██████████| 150000/150000 [03:53<00:00, 643.62it/s]


训练时间: 233.06 秒
平均GPU使用率: 35.87%
平均GPU显存使用量: 1993.00MiB
当前为第1次循环，种子为0


Adam: 100%|██████████| 150000/150000 [03:55<00:00, 636.60it/s]


训练时间: 235.64 秒
平均GPU使用率: 35.22%
平均GPU显存使用量: 1993.00MiB
当前为第2次循环，种子为1


Adam: 100%|██████████| 150000/150000 [03:53<00:00, 641.03it/s]


...

训练时间: 234.78 秒
平均GPU使用率: 35.65%
平均GPU显存使用量: 1993.00MiB
当前为第8次循环，种子为4321


Adam: 100%|██████████| 150000/150000 [03:55<00:00, 636.02it/s]


训练时间: 235.85 秒
平均GPU使用率: 35.43%
平均GPU显存使用量: 1993.00MiB
当前为第9次循环，种子为12345


Adam: 100%|██████████| 150000/150000 [03:55<00:00, 637.48it/s]


训练时间: 235.31 秒
平均GPU使用率: 35.48%
平均GPU显存使用量: 1993.00MiB
当前为第10次循环，种子为54321


In [60]:
#普通PINN
# 打印PINN训练时间
print(f'PINN训练时间为{[f"{traingtime}%" for traingtime in PINN_training_time]}s')
# 打印平均PINN训练时间
print(f'平均PINN训练时间为{np.mean(PINN_training_time):.2f}s')

# 打印GPU使用率
print(f'GPU使用率为{[f"{usage}%" for usage in GPU_usage]}')
# 打印平均GPU使用率
print(f'平均GPU使用率为{np.mean(GPU_usage):.2f}%')

# 打印GPU显存使用率
print(f'GPU显存使用为{[f"{memory}MiB" for memory in GPU_memory]}')
# 打印平均GPU显存使用率
print(f'平均GPU显存使用为{np.mean(GPU_memory):.2f}MiB')

PINN训练时间为['233.06319284439087%', '235.63507986068726%', '234.00477266311646%', '237.12041354179382%', '230.66630625724792%', '233.6735053062439%', '230.9328076839447%', '234.77720189094543%', '235.84825682640076%', '235.30731654167175%']s
平均PINN训练时间为234.10s
GPU使用率为['35.8716814159292%', '35.215859030837%', '35.45575221238938%', '35.61572052401747%', '36.027027027027025%', '35.79203539823009%', '36.24215246636771%', '35.647577092511014%', '35.43421052631579%', '35.480176211453745%']
平均GPU使用率为35.68%
GPU显存使用为['1993.0MiB', '1993.0MiB', '1993.0MiB', '1993.0MiB', '1993.0MiB', '1993.0MiB', '1993.0MiB', '1993.0MiB', '1993.0MiB', '1993.0MiB']
平均GPU显存使用为1993.00MiB


In [1]:
#计算效率=总训练时间/平均GPU使用率
efficiency = 234.1 / 0.3568
print(f'LDP效率为{efficiency:.2f}')

LDP效率为656.11


In [2]:
#总训练次数
N_f = 300
nIter = 150000
total_training = N_f * nIter

# 用科学计数法表示
total_training_scientific = f"{total_training:.2e}"
print(f'LDP总训练次数为 {total_training_scientific}')

LDP总训练次数为 4.50e+07


In [61]:
#RAR-G方法，对1000个点，先选择10个点训练500次，然后每500次迭代重采样100个点，选出其中残差最大的10个点添加到训练点中；最后总共有1000个点，共训练10000次
seeds = [0, 1, 12, 33, 123, 321, 1234, 4321, 12345, 54321] #生成10个随机种子
# seeds = [0]
#设置噪声水平为0
noise = 0.0        

#读取真实解(基于MCS方法得到的解)
MCS_solution_df = pd.read_csv('MCS_non_homo_300.csv')
MCS_solution_df.columns = ['time', 'state_0', 'state_1', 'state_2']

MCS_solution_df.drop(labels='time', inplace=True, axis = 1)


N_u = 1
N_f = 300

#定义一个列表layers，其中包含了神经网络的层数和每一层的神经元数量
layers = [1, 50, 50, 3]

lb  = np.array([0.0])              # t的左边界
ub  = np.array([300.0])          # t的右边界

# Define the initial MSS state
X_u = np.array([[0]])
u = np.array([[1,0,0]])          # 初始真实状态
# X_f = np.linspace(lb, ub, N_f)     # 配位点位置，0~60000的5000个点


PINN_training_time = [] #创建一个空列表，用于存储PINN训练时间
GPU_usage = [] #创建一个空列表，用于存储GPU使用率
GPU_memory = [] #创建一个空列表，用于存储GPU显存占用


i = 0 #初始化i为0

for seed in seeds:
    set_seed(seed) #设置随机数种子

    timeused = [] #创建一个空列表，用于存储时间
    Usage = [] #创建一个空列表，用于存储使用率
    Memory = [] #创建一个空列表，用于存储显存占用

    N_f_1 = N_f // 100 #先拿50个点训练
    X_f = lb + (ub - lb) * lhs(1, N_f*2) #生成N_f个随机点
    indices = np.arange(X_f.shape[0])
    id = np.random.choice(indices, N_f_1, replace=False) #从indices中随机选择10个数，replace=False表示不允许重复选择，最后将这10个数赋值给id，代表训练过的数据索引
    X_f_train = X_f[id, :] #从X_f_train中选取id对应的的10行，赋值给X_f_train


    nIter = 150000 #设置迭代次数为10000

    #创建PINN模型并输入各种参数     
    model = PhysicsInformedNN(X_u, u, X_f_train, layers, lb, ub)

    #开始训练模型            
    # model.train(nIter//100,0)
    training_time, avg_gpu_usage = train_model(model, nIter//100)

    timeused.append(training_time)
    Usage.append(avg_gpu_usage[0])
    Memory.append(avg_gpu_usage[1])

    #删除已经训练过的数据的索引
    indices = np.setdiff1d(indices, id) #从indices中去除id中的元素，最后将结果赋值给indices


    for iter in range(nIter//100+1, nIter+1, nIter//100): #每800次迭代
        N_f_new = N_f_1 #重新采样500个点
        # 生成新的X_f_train数据
        id = np.random.choice(indices, N_f_new, replace=False) #从indices中随机选择N_f_new个数，replace=False表示不允许重复选择，最后将这N_f_new个数赋值给id，代表训练过的数据索引

        X_f_train_new = X_f[id, :] #从X_f_train中选取id对应的的N_f_new行，赋值给X_f_train_new

        X_f_train = np.vstack((X_f_train, X_f_train_new)) #与之前的训练数据合并

        # 更新模型中的X_f_train数据
        model.update_data(X_u, u, X_f_train)

        # 在更新数据后的模型上进行训练500次
        # model.train(nIter//100,0)
        training_time, avg_gpu_usage = train_model(model, nIter//100)

        timeused.append(training_time)
        Usage.append(avg_gpu_usage[0])
        Memory.append(avg_gpu_usage[1])

        # 删除已经训练过的数据的索引
        indices = np.setdiff1d(indices, id)


    # print(X_f_train.shape)
    # 更新模型中的X_f_train数据
    model.update_data(X_u, u, X_f_train)

    timeused.append(training_time)
    Usage.append(avg_gpu_usage[0])
    Memory.append(avg_gpu_usage[1])

    training_time = sum(timeused)
    avg_gpu_usage = [np.mean(Usage), np.mean(Memory)]
    print(f"训练时间: {training_time:.2f} 秒")
    print(f"平均GPU使用率: {avg_gpu_usage[0]:.2f}%")
    print(f"平均GPU显存使用: {avg_gpu_usage[1]:.2f}MiB")

    PINN_training_time.append(training_time)
    GPU_usage.append(avg_gpu_usage[0])
    GPU_memory.append(avg_gpu_usage[1])


    i+=1 #i加1
    print(f'当前为第{i}次循环，种子为{seed}')


Adam: 100%|██████████| 1500/1500 [00:02<00:00, 626.03it/s]


训练时间: 247.54 秒
平均GPU使用率: 23.30%
平均GPU显存使用: 1993.00MiB
当前为第1次循环，种子为0


Adam: 100%|██████████| 1500/1500 [00:02<00:00, 577.75it/s]


训练时间: 249.37 秒
平均GPU使用率: 23.77%
平均GPU显存使用: 1993.00MiB
当前为第2次循环，种子为1


Adam: 100%|██████████| 1500/1500 [00:02<00:00, 615.53it/s]


...

训练时间: 248.73 秒
平均GPU使用率: 23.41%
平均GPU显存使用: 1993.00MiB
当前为第8次循环，种子为4321


Adam: 100%|██████████| 1500/1500 [00:02<00:00, 596.50it/s]


训练时间: 250.92 秒
平均GPU使用率: 23.05%
平均GPU显存使用: 1993.00MiB
当前为第9次循环，种子为12345


Adam: 100%|██████████| 1500/1500 [00:02<00:00, 603.84it/s]


训练时间: 248.62 秒
平均GPU使用率: 23.11%
平均GPU显存使用: 1993.00MiB
当前为第10次循环，种子为54321


In [62]:
#普通PINN
# 打印PINN训练时间
print(f'PINN训练时间为{[f"{traingtime}%" for traingtime in PINN_training_time]}s')
# 打印平均PINN训练时间
print(f'平均PINN训练时间为{np.mean(PINN_training_time):.2f}s')

# 打印GPU使用率
print(f'GPU使用率为{[f"{usage}%" for usage in GPU_usage]}')
# 打印平均GPU使用率
print(f'平均GPU使用率为{np.mean(GPU_usage):.2f}%')

# 打印GPU显存使用率
print(f'GPU显存使用为{[f"{memory}MiB" for memory in GPU_memory]}')
# 打印平均GPU显存使用率
print(f'平均GPU显存使用为{np.mean(GPU_memory):.2f}MiB')

PINN训练时间为['247.53658747673035%', '249.37459659576416%', '248.73502731323242%', '252.29371333122253%', '249.57435393333435%', '248.08062410354614%', '249.74359798431396%', '248.72997784614563%', '250.91997861862183%', '248.6161663532257%']s
平均PINN训练时间为249.36s
GPU使用率为['23.3036303630363%', '23.768976897689768%', '23.17821782178217%', '23.656765676567655%', '23.382838283828384%', '23.702970297029697%', '23.597359735973598%', '23.41254125412541%', '23.04950495049505%', '23.105610561056114%']
平均GPU使用率为23.42%
GPU显存使用为['1993.0MiB', '1993.0MiB', '1993.0MiB', '1993.0MiB', '1993.0MiB', '1993.0MiB', '1993.0MiB', '1993.0MiB', '1993.0MiB', '1993.0MiB']
平均GPU显存使用为1993.00MiB


In [3]:
#计算效率=总训练时间/平均GPU使用率
efficiency = 249.36 / 0.2342
print(f'NAP效率为{efficiency:.2f}')

NAP效率为1064.73


In [4]:
#总训练次数
def calculate_total_training(N_f, nIter):
    total_training = 0
    current_points = N_f // 100
    iterations_per_stage = nIter // 100

    for stage in range(1, 101):
        total_training += current_points * iterations_per_stage
        current_points += N_f // 100

    return total_training


total_training = calculate_total_training(N_f, nIter) 

# 用科学计数法表示
total_training_scientific = f"{total_training:.2e}"
print(f'NAP总训练次数为 {total_training_scientific}')

NAP总训练次数为 2.27e+07


In [65]:
#RAR-G方法，对1000个点，先选择10个点训练500次，然后每500次迭代重采样100个点，选出其中残差最大的10个点添加到训练点中；最后总共有1000个点，共训练10000次
seeds = [0, 1, 12, 33, 123, 321, 1234, 4321, 12345, 54321] #生成10个随机种子
# seeds = [0]
#设置噪声水平为0
noise = 0.0        

#读取真实解(基于MCS方法得到的解)
MCS_solution_df = pd.read_csv('MCS_non_homo_300.csv')
MCS_solution_df.columns = ['time', 'state_0', 'state_1', 'state_2']

MCS_solution_df.drop(labels='time', inplace=True, axis = 1)


N_u = 1
N_f = 300

#定义一个列表layers，其中包含了神经网络的层数和每一层的神经元数量
layers = [1, 50, 50, 3]

lb  = np.array([0.0])              # t的左边界
ub  = np.array([300.0])          # t的右边界

# Define the initial MSS state
X_u = np.array([[0]])
u = np.array([[1,0,0]])          # 初始真实状态
# X_f = np.linspace(lb, ub, N_f)     # 配位点位置，0~60000的5000个点


PINN_training_time = [] #创建一个空列表，用于存储PINN训练时间
GPU_usage = [] #创建一个空列表，用于存储GPU使用率
GPU_memory = [] #创建一个空列表，用于存储GPU显存占用


i = 0 #初始化i为0


for seed in seeds:
    set_seed(seed) #设置随机数种子

    timeused = [] #创建一个空列表，用于存储时间
    Usage = [] #创建一个空列表，用于存储使用率
    Memory = [] #创建一个空列表，用于存储显存占用


    N_f_1 = N_f // 100 #先拿50个点训练
    X_f_train = lb + (ub - lb) * lhs(1, N_f_1) #生成N_f个随机点
    nIter = 150000 #设置迭代次数为10000

    #创建PINN模型并输入各种参数     
    model = PhysicsInformedNN(X_u, u, X_f_train, layers, lb, ub)


    training_time, avg_gpu_usage = train_model(model, nIter//100)

    timeused.append(training_time)
    Usage.append(avg_gpu_usage[0])
    Memory.append(avg_gpu_usage[1])


    for iter in range(nIter//100+1, nIter+1, nIter//100): #每800次迭代
        N_f_new = N_f_1 * 10 #重新采样500个点
        X_f_train_new = lb + (ub - lb) * lhs(1, N_f_new) #生成N_f_new个随机点

        #计算残差
        _, residual = model.predict(X_f_train_new)
        abs_residual = np.abs(residual)
        abs_residual = abs_residual.flatten()
        #对abs_residual进行归一化
        abs_residual = abs_residual / np.linalg.norm(abs_residual)



        #计算信息量
        xinxi = abs_residual

        #选出信息量最大的10个点
        # 找出绝对值最大的10个值的索引
        topk_indices = np.argpartition(xinxi, -N_f_1)[-N_f_1:] #该函数会对数组进行排序，使得指定的k个最大值出现在数组的最后k给位置上，并获取最后1000个元素

        # 使用这些索引来提取对应的数据
        X_f_train_topk = X_f_train_new[topk_indices]

        X_f_train = np.vstack((X_f_train, X_f_train_topk)) #与之前的训练数据合并

        # 更新模型中的X_f_train数据
        model.update_data(X_u, u, X_f_train)

        # 在更新数据后的模型上进行训练500次
        # model.train(nIter//100,0)
        training_time, avg_gpu_usage = train_model(model, nIter//100)

        timeused.append(training_time)
        Usage.append(avg_gpu_usage[0])
        Memory.append(avg_gpu_usage[1])

    # print(X_f_train.shape)
    # 更新模型中的X_f_train数据
    model.update_data(X_u, u, X_f_train)


    training_time = sum(timeused)
    avg_gpu_usage = [np.mean(Usage), np.mean(Memory)]
    print(f"训练时间: {training_time:.2f} 秒")
    print(f"平均GPU使用率: {avg_gpu_usage[0]:.2f}%")
    print(f"平均GPU显存使用: {avg_gpu_usage[1]:.2f}MiB")

    PINN_training_time.append(training_time)
    GPU_usage.append(avg_gpu_usage[0])
    GPU_memory.append(avg_gpu_usage[1])

    

    i+=1 #i加1
    print(f'当前为第{i}次循环，种子为{seed}')


Adam: 100%|██████████| 1500/1500 [00:02<00:00, 643.14it/s]


训练时间: 243.88 秒
平均GPU使用率: 23.82%
平均GPU显存使用: 1993.00MiB
当前为第1次循环，种子为0


Adam: 100%|██████████| 1500/1500 [00:02<00:00, 615.14it/s]


训练时间: 245.10 秒
平均GPU使用率: 23.76%
平均GPU显存使用: 1993.00MiB
当前为第2次循环，种子为1


Adam: 100%|██████████| 1500/1500 [00:02<00:00, 595.63it/s]


...

训练时间: 245.92 秒
平均GPU使用率: 23.48%
平均GPU显存使用: 1993.00MiB
当前为第8次循环，种子为4321


Adam: 100%|██████████| 1500/1500 [00:02<00:00, 622.96it/s]


训练时间: 245.21 秒
平均GPU使用率: 23.57%
平均GPU显存使用: 1993.00MiB
当前为第9次循环，种子为12345


Adam: 100%|██████████| 1500/1500 [00:02<00:00, 565.72it/s]


训练时间: 247.48 秒
平均GPU使用率: 23.25%
平均GPU显存使用: 1993.00MiB
当前为第10次循环，种子为54321


In [66]:
#fuhe PINN
# 打印PINN训练时间
print(f'PINN训练时间为{[f"{traingtime}%" for traingtime in PINN_training_time]}s')
# 打印平均PINN训练时间
print(f'平均PINN训练时间为{np.mean(PINN_training_time):.2f}s')

# 打印GPU使用率
print(f'GPU使用率为{[f"{usage}%" for usage in GPU_usage]}')
# 打印平均GPU使用率
print(f'平均GPU使用率为{np.mean(GPU_usage):.2f}%')

# 打印GPU显存使用率
print(f'GPU显存使用为{[f"{memory}MiB" for memory in GPU_memory]}')
# 打印平均GPU显存使用率
print(f'平均GPU显存使用为{np.mean(GPU_memory):.2f}MiB')



PINN训练时间为['243.87964487075806%', '245.09909915924072%', '244.53341245651245%', '243.26569533348083%', '245.29916834831238%', '244.61933207511902%', '244.79279446601868%', '245.92412066459656%', '245.21171760559082%', '247.47531080245972%']s
平均PINN训练时间为245.01s
GPU使用率为['23.82333333333333%', '23.76%', '23.463333333333335%', '23.526666666666674%', '23.48333333333333%', '23.51%', '23.509999999999994%', '23.479999999999997%', '23.5675%', '23.246666666666666%']
平均GPU使用率为23.54%
GPU显存使用为['1993.0MiB', '1993.0MiB', '1993.0MiB', '1993.0MiB', '1993.0MiB', '1993.0MiB', '1993.0MiB', '1993.0MiB', '1993.0MiB', '1993.0MiB']
平均GPU显存使用为1993.00MiB


In [5]:
#计算效率=总训练时间/平均GPU使用率
efficiency = 245.01 / 0.2354
print(f'RSP效率为{efficiency:.2f}')

RSP效率为1040.82


In [6]:
#总训练次数
def calculate_total_training(N_f, nIter):
    total_training = 0
    current_points = N_f // 100
    iterations_per_stage = nIter // 100

    for stage in range(1, 101):
        total_training += current_points * iterations_per_stage
        current_points += N_f // 100

    return total_training


total_training = calculate_total_training(N_f, nIter) 

# 用科学计数法表示
total_training_scientific = f"{total_training:.2e}"
print(f'RSP总训练次数为 {total_training_scientific}')

RSP总训练次数为 2.27e+07


In [67]:
#RAR-G方法，对1000个点，先选择10个点训练500次，然后每500次迭代重采样100个点，选出其中残差最大的10个点添加到训练点中；最后总共有1000个点，共训练10000次
seeds = [0, 1, 12, 33, 123, 321, 1234, 4321, 12345, 54321] #生成10个随机种子
# seeds = [0]
#设置噪声水平为0
noise = 0.0        

#读取真实解(基于MCS方法得到的解)
MCS_solution_df = pd.read_csv('MCS_non_homo_300.csv')
MCS_solution_df.columns = ['time', 'state_0', 'state_1', 'state_2']

MCS_solution_df.drop(labels='time', inplace=True, axis = 1)


N_u = 1
N_f = 300

#定义一个列表layers，其中包含了神经网络的层数和每一层的神经元数量
layers = [1, 50, 50, 3]

lb  = np.array([0.0])              # t的左边界
ub  = np.array([300.0])          # t的右边界

# Define the initial MSS state
X_u = np.array([[0]])
u = np.array([[1,0,0]])          # 初始真实状态
# X_f = np.linspace(lb, ub, N_f)     # 配位点位置，0~60000的5000个点


PINN_training_time = [] #创建一个空列表，用于存储PINN训练时间
GPU_usage = [] #创建一个空列表，用于存储GPU使用率
GPU_memory = [] #创建一个空列表，用于存储GPU显存占用


i = 0 #初始化i为0


for seed in seeds:
    set_seed(seed) #设置随机数种子

    timeused = [] #创建一个空列表，用于存储时间
    Usage = [] #创建一个空列表，用于存储使用率
    Memory = [] #创建一个空列表，用于存储显存占用


    N_f_1 = N_f // 100 #先拿50个点训练
    X_f_train = lb + (ub - lb) * lhs(1, N_f_1) #生成N_f个随机点
    nIter = 150000 #设置迭代次数为10000

    #创建PINN模型并输入各种参数     
    model = PhysicsInformedNN(X_u, u, X_f_train, layers, lb, ub)


    training_time, avg_gpu_usage = train_model(model, nIter//100)

    timeused.append(training_time)
    Usage.append(avg_gpu_usage[0])
    Memory.append(avg_gpu_usage[1])


    for iter in range(nIter//100+1, nIter+1, nIter//100): #每800次迭代
        N_f_new = N_f_1 * 10 #重新采样500个点
        X_f_train_new = lb + (ub - lb) * lhs(1, N_f_new) #生成N_f_new个随机点

        #计算残差
        _, residual = model.predict(X_f_train_new)
        abs_residual = np.abs(residual)
        abs_residual = abs_residual.flatten()
        #对abs_residual进行归一化
        abs_residual = abs_residual / np.linalg.norm(abs_residual)


        #计算混沌度
        y_chaos = model.hidden_predict(X_f_train_new, 100)
        x_ssss = X_f_train_new + np.random.normal(0, 0.001)
        y_chaos_ssss = model.hidden_predict(x_ssss, 100)
        chaos = np.linalg.norm(y_chaos - y_chaos_ssss,axis=1)
        #对chaos进行归一化
        epsilon = 1e-10
        norm_chaos = np.linalg.norm(chaos)
        if norm_chaos > epsilon:
            chaos = chaos / norm_chaos
        else:
            chaos = np.zeros_like(chaos)


        #计算信息量
        xinxi = 0.7*abs_residual + 0.3*chaos

        #选出信息量最大的10个点
        # 找出绝对值最大的10个值的索引
        topk_indices = np.argpartition(xinxi, -N_f_1)[-N_f_1:] #该函数会对数组进行排序，使得指定的k个最大值出现在数组的最后k给位置上，并获取最后1000个元素

        # 使用这些索引来提取对应的数据
        X_f_train_topk = X_f_train_new[topk_indices]

        X_f_train = np.vstack((X_f_train, X_f_train_topk)) #与之前的训练数据合并

        # 更新模型中的X_f_train数据
        model.update_data(X_u, u, X_f_train)

        # 在更新数据后的模型上进行训练500次
        # model.train(nIter//100,0)
        training_time, avg_gpu_usage = train_model(model, nIter//100)

        timeused.append(training_time)
        Usage.append(avg_gpu_usage[0])
        Memory.append(avg_gpu_usage[1])

    # print(X_f_train.shape)
    # 更新模型中的X_f_train数据
    model.update_data(X_u, u, X_f_train)


    training_time = sum(timeused)
    avg_gpu_usage = [np.mean(Usage), np.mean(Memory)]
    print(f"训练时间: {training_time:.2f} 秒")
    print(f"平均GPU使用率: {avg_gpu_usage[0]:.2f}%")
    print(f"平均GPU显存使用: {avg_gpu_usage[1]:.2f}MiB")

    PINN_training_time.append(training_time)
    GPU_usage.append(avg_gpu_usage[0])
    GPU_memory.append(avg_gpu_usage[1])

    

    i+=1 #i加1
    print(f'当前为第{i}次循环，种子为{seed}')


Adam: 100%|██████████| 1500/1500 [00:02<00:00, 525.67it/s]


训练时间: 241.17 秒
平均GPU使用率: 23.61%
平均GPU显存使用: 1993.00MiB
当前为第1次循环，种子为0


Adam: 100%|██████████| 1500/1500 [00:02<00:00, 578.66it/s]


训练时间: 242.82 秒
平均GPU使用率: 23.02%
平均GPU显存使用: 1993.00MiB
当前为第2次循环，种子为1


Adam: 100%|██████████| 1500/1500 [00:02<00:00, 645.74it/s]


...

训练时间: 244.35 秒
平均GPU使用率: 23.66%
平均GPU显存使用: 1993.00MiB
当前为第8次循环，种子为4321


Adam: 100%|██████████| 1500/1500 [00:02<00:00, 628.10it/s]


训练时间: 243.91 秒
平均GPU使用率: 23.15%
平均GPU显存使用: 1993.00MiB
当前为第9次循环，种子为12345


Adam: 100%|██████████| 1500/1500 [00:02<00:00, 560.16it/s]


训练时间: 244.34 秒
平均GPU使用率: 23.64%
平均GPU显存使用: 1993.00MiB
当前为第10次循环，种子为54321


In [68]:
#fuhe PINN
# 打印PINN训练时间
print(f'PINN训练时间为{[f"{traingtime}%" for traingtime in PINN_training_time]}s')
# 打印平均PINN训练时间
print(f'平均PINN训练时间为{np.mean(PINN_training_time):.2f}s')

# 打印GPU使用率
print(f'GPU使用率为{[f"{usage}%" for usage in GPU_usage]}')
# 打印平均GPU使用率
print(f'平均GPU使用率为{np.mean(GPU_usage):.2f}%')

# 打印GPU显存使用率
print(f'GPU显存使用为{[f"{memory}MiB" for memory in GPU_memory]}')
# 打印平均GPU显存使用率
print(f'平均GPU显存使用为{np.mean(GPU_memory):.2f}MiB')



PINN训练时间为['241.1663315296173%', '242.82430124282837%', '245.14883732795715%', '245.1964807510376%', '245.6809802055359%', '242.8614284992218%', '244.7950508594513%', '244.35328674316406%', '243.90965962409973%', '244.33927202224731%']s
平均PINN训练时间为244.03s
GPU使用率为['23.61000000000001%', '23.016666666666666%', '23.446666666666665%', '23.063333333333336%', '23.49666666666667%', '23.14%', '23.483333333333334%', '23.65666666666667%', '23.15%', '23.635%']
平均GPU使用率为23.37%
GPU显存使用为['1993.0MiB', '1993.0MiB', '1993.0MiB', '1993.0MiB', '1993.0MiB', '1993.0MiB', '1993.0MiB', '1993.0MiB', '1993.0MiB', '1993.0MiB']
平均GPU显存使用为1993.00MiB


In [7]:
#计算效率=总训练时间/平均GPU使用率
efficiency = 244.03 / 0.2337
print(f'RCP效率为{efficiency:.2f}')

RCP效率为1044.20


In [8]:
#总训练次数
def calculate_total_training(N_f, nIter):
    total_training = 0
    current_points = N_f // 100
    iterations_per_stage = nIter // 100

    for stage in range(1, 101):
        total_training += current_points * iterations_per_stage
        current_points += N_f // 100

    return total_training


total_training = calculate_total_training(N_f, nIter) 

# 用科学计数法表示
total_training_scientific = f"{total_training:.2e}"
print(f'RCP总训练次数为 {total_training_scientific}')

RCP总训练次数为 2.27e+07
